In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [39]:
# Set features. This will also be used as your x values.
# selected_features = df[['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr']]
selected_features = df.drop(["koi_disposition",'koi_tce_plnt_num', 'koi_srad_err2', 'koi_slogg_err1',
                             'koi_steff', 'koi_srad', 'koi_slogg'], axis=1)
# selected_features = df.drop("koi_disposition", axis=1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [40]:
y = df["koi_disposition"]
print(selected_features.shape, y.shape)

(6991, 34) (6991,)


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, random_state=1)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_steff_err1,koi_steff_err2,koi_slogg_err2,koi_srad_err1,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,5.470000e-05,-5.470000e-05,139.064020,0.004110,-0.004110,...,72.78,-37.23,11.7,120,-133,-0.123,0.181,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,1.365000e-04,-1.365000e-04,140.207320,0.004460,-0.004460,...,3.92,-3.75,18.0,144,-144,-0.052,0.056,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,1.230000e-07,-1.230000e-07,131.792007,0.000096,-0.000096,...,184.18,-164.28,476.0,126,-140,-0.027,0.054,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,1.461000e-03,-1.461000e-03,187.569860,0.005290,-0.005290,...,0.77,-0.44,34.7,101,-112,-0.108,0.135,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,3.181000e-03,-3.181000e-03,175.715600,0.028600,-0.028600,...,41.53,-12.19,8.7,164,-233,-0.164,0.905,294.93198,39.812420,12.964


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [42]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Label-encode data
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Train the Model



In [43]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train_scaled, encoded_y_train)
predictions = model.predict(X_test_scaled)
print(predictions)

print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, encoded_y_test)}")

[1 2 2 ... 2 2 0]
Training Data Score: 1.0
Testing Data Score: 0.9153318077803204


In [44]:
feature_names = selected_features.columns
sorted(zip(model.feature_importances_,feature_names), reverse=True)

[(0.10748034380624452, 'koi_fpflag_co'),
 (0.09299071872924658, 'koi_fpflag_nt'),
 (0.07050757631237518, 'koi_fpflag_ss'),
 (0.0605779497899023, 'koi_model_snr'),
 (0.04381459688203698, 'koi_prad'),
 (0.04046413132609673, 'koi_prad_err2'),
 (0.03582377325760293, 'koi_duration_err1'),
 (0.03479224770411839, 'koi_prad_err1'),
 (0.03407249819747255, 'koi_fpflag_ec'),
 (0.031881710798249745, 'koi_duration_err2'),
 (0.03133182578474429, 'koi_steff_err2'),
 (0.03056285158369585, 'koi_steff_err1'),
 (0.025001296319958254, 'koi_duration'),
 (0.024570986906739256, 'koi_time0bk_err1'),
 (0.023597685674924606, 'koi_time0bk_err2'),
 (0.02261003540336695, 'koi_period'),
 (0.021770691338013844, 'koi_insol_err1'),
 (0.021672337936640224, 'koi_depth'),
 (0.020298858662205307, 'koi_period_err1'),
 (0.020031170106304473, 'koi_period_err2'),
 (0.019644621027267536, 'koi_impact'),
 (0.019251517413446207, 'koi_teq'),
 (0.016386167099337864, 'koi_time0bk'),
 (0.015812257431042413, 'koi_depth_err2'),
 (0.014

In [45]:
# Classification report
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions, 
                            target_names = ["Confirmed", "False Positive", "Candidate"]))

                precision    recall  f1-score   support

     Confirmed       0.85      0.77      0.81       404
False Positive       0.82      0.87      0.84       435
     Candidate       0.99      1.00      0.99       909

      accuracy                           0.92      1748
     macro avg       0.89      0.88      0.88      1748
  weighted avg       0.92      0.92      0.91      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [46]:
# Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion': ['gini']}
grid = GridSearchCV(model, param_grid, verbose=0)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# Save the Model

In [14]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'RndForest_model.sav'
joblib.dump(model, filename)

['RndForest_model.sav']